# Checkers

## Rules
<ul>
<li>Two players</li>
<li>8 by 8 board</li>
<li>Dark and light squares alternate</li>
<li>12 pieces per side</li>
<li>Pieces can only be placed on dark squares (light squares need to remain empty)</li>
<li>Players alternate turns</li>
<li>Players can only move their own pieces</li>
<li>Players can only move one piece per move</li>
<li>Moves include:</li>
<ul>
<li>Simple move: 1 step diagonally to an unoccupied adjacent square</li>
<li>Jump: 'Jumping' diagonally over an occupied to an unoccupied adjacent square</li>
<li>Jumping leads to the jumped piece being 'captured' and removed from the game</li>
<li>Jumping is mandatory</li>
<li>Multi-jumps are possible and can happen in different directions</li>
<li>Multi-jump sequences need to be completed</li>
<li>Man can only move forward, kings foward and backward</li>
</ul>
<li>Two classes of pieces: Men and kings</li>
<ul>
<li>Men (one piece)</li>
<li>Kings (two pieces stacked on top of each other)</li>
<li>Kings are 'crowned' when pieces have reached the opposite site's kings row</li>
<li>Reaching the opposite site's kings row terminates a move</li>
</ul>
<li>The game ends with a winner when one player has captured all of the other player's pieces</li> 
<li>The game ends with a winner when one player has no more legal moves available.
<li>The game ends in a draw when neither player can achieve a win</li>
</ul>

In [1]:
import copy

In [12]:
class Game:    
    def __init__(self):
        self.status = "In progress"

        self.state = [
            [0, 'wm4', 0, 'wm3', 0, 'wm2', 0, 'wm1'],
            ['wm8', 0, 'wm7', 0, 'wm6', 0, 'wm5', 0],
            [0, 'wm12', 0, 'wm11', 0, 'wm10', 0, 'wm9'],
            [1, 0, 1, 0, 1, 0, 1, 0],
            [0, 1, 0, 1, 0, 1, 0, 1],
            ['rm9', 0, 'rm10', 0, 'rm11', 0, 'rm12', 0],
            [0, 'rm5', 0, 'rm6', 0, 'rm7', 0, 'rm8'],
            ['rm1', 0, 'rm2', 0, 'rm3', 0, 'rm4', 0]
        ]

    def count_white(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('wm') + 2*flat_state.count('wk')

    def count_red(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('rm') + 2*flat_state.count('rk')
    
    def count_white_kings(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('wk')

    def count_red_kings(self):
        flat_state = [field for row in self.state for field in row]
        return flat_state.count('rk')

    def get_total_distance_to_k_row_white(self):
            total_distance_to_k_row_w = 0
            for row_id, row in enumerate(self.state):
                for field in row:
                    if field != 0 and field != 1:
                        if 'w' in field:
                            distance_to_k_row_w = 7-row_id
                            total_distance_to_k_row_w += distance_to_k_row_w
            return total_distance_to_k_row_w
            
    def get_total_distance_to_k_row_red(self):
            total_distance_to_k_row_r = 0
            for row_id, row in enumerate(self.state):
                for field in row:
                    if field != 0 and field != 1:
                        if 'r' in field:
                            distance_to_k_row_r = row_id
                            total_distance_to_k_row_r += distance_to_k_row_r
            return total_distance_to_k_row_r

    def get_score(self):
        # Evaluation function taking into account checker count delta, king count delta, and 
        # the delta of the total distance of all checkers from the kings row
        count_delta = self.count_white() - self.count_red()
        king_delta = self.count_white_kings() - self.count_red_kings()
        k_row_distance_delta = self.get_total_distance_to_k_row_white() - self.get_total_distance_to_k_row_red()
        # Applying an arbitrary weighting (based on the distance delta being expected to be a higher magnitude)
        return 0.5*count_delta + 0.45*king_delta + 0.05*k_row_distance_delta
        # Simple evaluation function based purely on checker count delta
        # return self.count_white() - self.count_red()

    def who_won(self):
        score = self.get_score()
        if score > 0:
            return 'White wins'
        elif score < 0:
            return 'Red wins'
        else:
            return "Draw"
        
    def update_state(self, state):
        if not state:
            self.update_status("Won")
            return
        self.state = state
        self.update_status()

    def update_status(self, status=None):
        if status:
            self.status = status
        if self.count_white == 0 or self.count_red == 0:
            self.status = "Won"
            
    def can_make_move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        if vertical_direction == "backward" and 'k' not in checker:
            return False
            
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if self.location_is_on_board(next_checker_row, next_checker_col):
            next_location_value = state[next_checker_row][next_checker_col]
            return next_location_value == 1

    def move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        king_row = 7
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
            king_row = 0
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if next_checker_row == king_row:
            checker = checker[0] + 'k'

        next_state = copy.deepcopy(state)
        next_state[next_checker_row][next_checker_col] = checker
        next_state[initial_checker_location[0]][initial_checker_location[1]] = 1 

        return next_state



In [16]:
class Player:
    def __init__(self, game, colour):
        self.game = game
        self.colour = colour        
        
    def find_best_move(self, state, depth, alpha, beta, max_player):
        """
        Implements minimax with alpha-beta pruning
        """
        if depth == 0:
            return self.game.get_score(), None
        
        # Maximizing player
        if max_player == True:
            max_evaluation = -1000
            best_next_state = None
            next_possible_states = self._get_next_possible_states(state, True)
            if len(next_possible_states) == 0:
                return max_evaluation, None
            for possible_state in next_possible_states:
                curr_evaluation, _ = self.find_best_move(possible_state, depth - 1, alpha, beta, False)
                max_evaluation = max(max_evaluation, curr_evaluation)
                if max_evaluation == curr_evaluation:
                    best_next_state = possible_state
                alpha = max(alpha, curr_evaluation)
                if beta <= alpha:
                    break
            return max_evaluation, best_next_state
        else:
            # Minimizing player
            min_evaluation = 1000
            best_next_state = None
            next_possible_states = self._get_next_possible_states(state, False)
            if len(next_possible_states) == 0:
                return min_evaluation, None
            for possible_state in next_possible_states:
                curr_evaluation, _ = self.find_best_move(possible_state, depth - 1, alpha, beta, True)
                min_evaluation = min(min_evaluation, curr_evaluation)
                if min_evaluation == curr_evaluation:
                    best_next_state = possible_state
                beta = min(beta, curr_evaluation)
                if beta <= alpha:
                    break
            return min_evaluation, best_next_state

    def _get_next_possible_states(self, state, max_player):
        next_states = []
        for row_id, row in enumerate(state):
            for col_id, field in enumerate(row):
                if field != 0 and field != 1 and self.colour in field:
                    if self.can_make_move('forward', 'left', field, [row_id, col_id], state):
                        next_states.append(self.move('forward', 'left', field, [row_id, col_id], state))
                    if self.can_make_move('forward', 'right', field, [row_id, col_id], state):
                        next_states.append(self.move('forward', 'right', field, [row_id, col_id], state))
                    if self.can_make_move('backward', 'left', field, [row_id, col_id], state):
                        next_states.append(self.move('backward', 'left', field, [row_id, col_id], state))
                    if self.can_make_move('backward', 'right', field, [row_id, col_id], state):
                        next_states.append(self.move('backward', 'right', field, [row_id, col_id], state))

                    if self.can_make_jump('forward', 'left',field, [row_id, col_id], state):
                        next_states.append(self.jump('forward', 'left', field, [row_id, col_id], state))
                    if self.can_make_jump('forward', 'right',field, [row_id, col_id], state):
                        next_states.append(self.jump('forward', 'right', field, [row_id, col_id], state))
                    if self.can_make_jump('backward', 'left',field, [row_id, col_id], state):
                        next_states.append(self.jump('backward', 'left', field, [row_id, col_id], state))
                    if self.can_make_jump('backward', 'right',field, [row_id, col_id], state):
                        next_states.append(self.jump('backward', 'right', field, [row_id, col_id], state))
        return next_states
                        
    def can_make_move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        if vertical_direction == "backward" and 'k' not in checker:
            return False
            
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if self.location_is_on_board(next_checker_row, next_checker_col):
            next_location_value = state[next_checker_row][next_checker_col]
            return next_location_value == 1

    def move(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        king_row = 7
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
            king_row = 0
        
        next_checker_row = initial_checker_location[0] + vertical_move
        next_checker_col = initial_checker_location[1] + horizontal_move

        if next_checker_row == king_row:
            checker = checker[0] + 'k'

        next_state = copy.deepcopy(state)
        next_state[next_checker_row][next_checker_col] = checker
        next_state[initial_checker_location[0]][initial_checker_location[1]] = 1 

        return next_state


    def can_make_jump(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        if vertical_direction == "backward" and 'k' not in checker:
            return False
        
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
        
        next_jump_row = initial_checker_location[0] + vertical_move
        next_land_row = initial_checker_location[0] + 2*vertical_move
        next_jump_col = initial_checker_location[1] + horizontal_move
        next_land_col = initial_checker_location[1] + 2*horizontal_move

        if self.location_is_on_board(next_land_row, next_land_col):
            next_jump_value = state[next_jump_row][next_jump_col]
            next_land_value = state[next_land_row][next_land_col]
            return next_jump_value != 1 and not self.colour in next_jump_value and next_land_value == 1

    def jump(self, vertical_direction, horizontal_direction, checker, initial_checker_location, state):
        horizontal_move = -1 if horizontal_direction == 'left' else 1
        vertical_move = 1 if vertical_direction == 'forward' else -1
        king_row = 7
        if 'r' in checker:
            horizontal_move *= -1
            vertical_move *= -1
            king_row = 0
        
        next_jump_row = initial_checker_location[0] + vertical_move
        next_land_row = initial_checker_location[0] + 2*vertical_move
        next_jump_col = initial_checker_location[1] + horizontal_move
        next_land_col = initial_checker_location[1] + 2*horizontal_move

        if next_land_row == king_row:
            checker = checker[0] + 'k'

        next_state = copy.deepcopy(state)
        next_state[next_land_row][next_land_col] = checker
        next_state[next_jump_row][next_jump_col] = 1
        next_state[initial_checker_location[0]][initial_checker_location[1]] = 1
        
        return next_state
        

    def location_is_on_board(self, row, col):
        return row >= 0 and row <= 7 and col >= 0 and col <= 7


In [17]:
def print_state(state):

    print('\n \n'.join(['  '.join(["{:" ">3}".format(item) for item in row]) 
      for row in state]))
    
#     for row in state:
#         print(row) print('\n')

In [18]:
# ALGO VS ALGO

game = Game()
player_1 = Player(game, 'w')
player_2 = Player(game, 'r')

current_player = player_1
while game.status != "Won":
    max_player = True if current_player == player_1 else False
    _, next_state = current_player.find_best_move(game.state, 3, -1000, 1000, max_player)
    game.update_state(next_state)
    if current_player == player_1:
        current_player = player_2
    else:
        current_player = player_1

print_state(game.state)
print(game.who_won())
    


  0  wm4    0  wm3    0  wm2    0  wm1
 
wm8    0  wm7    0  wm6    0  wm5    0
 
  0  rm7    0  rm6    0  rm3    0    1
 
rm10    0  rm1    0  rm2    0   wk    0
 
  0  rm5    0    1    0    1    0    1
 
rm9    0    1    0    1    0    1    0
 
  0    1    0    1    0    1    0    1
 
  1    0    1    0    1    0    1    0
White wins


In [75]:
# HUMAN VS ALGO

# Init Game
game = Game()

# Define Game Players
ai_player = Player(game, 'r')
#human_player = TODO: Define a game player, create a new class?

# Assign the human as games first player
current_player = ai_player

# Loop while there is still no winner
# - AI is always playing as MAX
while game.status != "Won":
    
    # Let AI Take turn + update the game.state with this move
    next_state = current_player.find_best_move(game.state, 3, -1000, 1000, max_player)
    game.update_state(next_state)
    
    # Return and print the game_state
    print_state(game.state)
    
    # Retrieve User input
    # - What is the user input 
    #
    # Update board with user move
    # - Row/Col used to update value?
    #     - This would be a simple update function, moving once checker to another location
    # - How do we track the path taken by the checker AND signify which checkers have been jumped and taken
    #     - One solution is letting the user manually update the board
    #     - Can these existing functions be used for this?
    #
    # - How is the board updated, we need to:
    #    - Move checker
    #    - Remove any jumped checkers
    #
    # 
    # next_state = user input
    # game.update(next_state)
    
    # If current_player = human:
    #    current_player = AI
    # Else:
    #     current_player = human
    
    
    

TypeError: 'float' object is not iterable

In [ ]:
class Human:
    # Retreive User input
    selected_checker = input("Please enter the piece you would like to move: ")
    move_direction - input("Please enter the direction you would like to move: ")
    
    
    # Create game state with user input
    
    # Execute can_move and can_jump to validate the move
    
    # If valid, update game state
    
    # Else, Invalid Move
    